In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install skl2onnx

In [7]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00


## Data Load and Create ML Model

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

iris = load_iris()
X,y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

## Save Final Model

In [4]:
joblib.dump(clf, '/content/drive/MyDrive/ONNX/iris_model.pkl', compress=9)

['/content/drive/MyDrive/ONNX/iris_model.pkl']

## ML Model To ONNX Model Conversion

In [5]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import joblib

cls = joblib.load('/content/drive/MyDrive/ONNX/iris_model.pkl')
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(cls, initial_types=initial_type)

with open('/content/drive/MyDrive/ONNX/iris_model.onnx', 'wb') as f:
    f.write(onx.SerializeToString())

## Test Our ONNX Model

In [8]:
import onnxruntime as rt
import numpy as np

data = np.array([[4.5,4.9,5.1,5.4],[1.5,2.9,3.1,1.4],[7.5,6.9,8.1,6.4]])

sess = rt.InferenceSession("/content/drive/MyDrive/ONNX/iris_model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

pred_onnx = sess.run([label_name], {input_name: data.astype(np.float32)})[0]
print(pred_onnx)

[2 1 2]
